# Experiment_dataset

In [1]:
import pandas as pd
from dataset.data_manager import BuildDataset

from dataset_labeling_study import aggiorna_label_distanza_temporale
from dataset_labeling_study import create_tile_videos_relabeling, create_df_video_from_master_df_relabeling

/home/isac/miniconda3/envs/videomae/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
# NON carico quello con tutti i metodi  all_data_all_methods_tracks_complete_fast.csv

sup_data_train = BuildDataset(type='SUPERVISED', master_df_path="all_data_CL7_tracks_complete_fast.csv")
sup_data_train.load_master_df()

In [16]:
sup_data_train.master_df.dtypes

path             string[python]
datetime         datetime64[ns]
tile_offset_x             int16
tile_offset_y             int16
x_pix                    object
y_pix                    object
source           string[python]
id_cyc_unico              int32
start_time       datetime64[ns]
end_time         datetime64[ns]
label                      int8
dtype: object

In [17]:
sup_data_train.calc_delta_time()

In [18]:
sup_data_train.master_df

,path,datetime,tile_offset_x,tile_offset_y,x_pix,y_pix,source,id_cyc_unico,start_time,end_time,label,delta_time
0,../fromgcloud/airmass_rgb_20111101_0000.png,2011-11-01,0,0,[],[],[],0,NaT,NaT,0,NaT
1,../fromgcloud/airmass_rgb_20111101_0000.png,2011-11-01,213,0,[],[],[],0,NaT,NaT,0,NaT
2,../fromgcloud/airmass_rgb_20111101_0000.png,2011-11-01,426,0,[],[],[],0,NaT,NaT,0,NaT
3,../fromgcloud/airmass_rgb_20111101_0000.png,2011-11-01,639,0,[],[],[],0,NaT,NaT,0,NaT
4,../fromgcloud/airmass_rgb_20111101_0000.png,2011-11-01,852,0,[],[],[],0,NaT,NaT,0,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...
1413163,../fromgcloud/airmass_rgb_20230912_0000.png,2023-09-12,213,196,[],[],[],0,NaT,NaT,0,NaT
1413164,../fromgcloud/airmass_rgb_20230912_0000.png,2023-09-12,426,196,[],[],[],0,NaT,NaT,0,NaT
1413165,../fromgcloud/airmass_rgb_20230912_0000.png,2023-09-12,639,196,[],[],[],0,NaT,NaT,0,NaT
1413166,../fromgcloud/airmass_rgb_20230912_0000.png,2023-09-12,852,196,[],[],[],0,NaT,NaT,0,NaT


In [19]:
aggiorna_label_distanza_temporale(sup_data_train.master_df, soglia=pd.Timedelta(hours=12), sub_lab=-1)

Colonna label_12_00 aggiunta


'label_12_00'

In [ ]:
######################################
# # verifico che il relabeling sia corretto per la parte iniziale e finale della durata del ciclone
pd.set_option('display.max_rows', 500)
m = sup_data_train.master_df.label_12_00 == -1
moff = (sup_data_train.master_df[['tile_offset_x','tile_offset_y']] == (213,0)).all(axis=1)

m_cyc = sup_data_train.master_df.id_cyc_unico == 7001328
sup_data_train.master_df[m_cyc].loc[20545:23700] #14545:14600]

### ora levo le righe con label == -1 e poi prendo la colonna relabel come label ufficiale

In [28]:
m = sup_data_train.master_df.label_12_00 == -1
df_mod = sup_data_train.master_df[~m].copy().drop(columns='label').rename(columns={'label_12_00':'label'})

In [29]:
df_mod

,path,datetime,tile_offset_x,tile_offset_y,x_pix,y_pix,source,id_cyc_unico,start_time,end_time,delta_time,label
0,../fromgcloud/airmass_rgb_20111101_0000.png,2011-11-01,0,0,[],[],[],0,NaT,NaT,NaT,0
1,../fromgcloud/airmass_rgb_20111101_0000.png,2011-11-01,213,0,[],[],[],0,NaT,NaT,NaT,0
2,../fromgcloud/airmass_rgb_20111101_0000.png,2011-11-01,426,0,[],[],[],0,NaT,NaT,NaT,0
3,../fromgcloud/airmass_rgb_20111101_0000.png,2011-11-01,639,0,[],[],[],0,NaT,NaT,NaT,0
4,../fromgcloud/airmass_rgb_20111101_0000.png,2011-11-01,852,0,[],[],[],0,NaT,NaT,NaT,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1413163,../fromgcloud/airmass_rgb_20230912_0000.png,2023-09-12,213,196,[],[],[],0,NaT,NaT,NaT,0
1413164,../fromgcloud/airmass_rgb_20230912_0000.png,2023-09-12,426,196,[],[],[],0,NaT,NaT,NaT,0
1413165,../fromgcloud/airmass_rgb_20230912_0000.png,2023-09-12,639,196,[],[],[],0,NaT,NaT,NaT,0
1413166,../fromgcloud/airmass_rgb_20230912_0000.png,2023-09-12,852,196,[],[],[],0,NaT,NaT,NaT,0


In [30]:
sup_data_train.make_df_video(new_master_df=df_mod, output_dir=None,  is_to_balance=True)  #idxs=[1,2,3,4,5,6,7,8],

1)  ->
Num video CON cicloni: 104, Num video SENZA cicloni: 2018	Totale video tiles: 2122		Bilanciamento video...
 video senza cicloni tenuti: 104
208 video per il periodo (effettivo) da 2011-11-01 00:00:00 a 2011-11-10 22:10:00

2)  ->
Num video CON cicloni: 99, Num video SENZA cicloni: 2016	Totale video tiles: 2115		Bilanciamento video...
 video senza cicloni tenuti: 99
198 video per il periodo (effettivo) da 2014-11-03 01:20:00 a 2014-11-12 23:55:00

3)  ->
Num video CON cicloni: 88, Num video SENZA cicloni: 1622	Totale video tiles: 1710		Bilanciamento video...
 video senza cicloni tenuti: 88
176 video per il periodo (effettivo) da 2016-10-26 00:00:00 a 2016-11-02 23:55:00

4)  ->
Num video CON cicloni: 83, Num video SENZA cicloni: 1618	Totale video tiles: 1701		Bilanciamento video...
 video senza cicloni tenuti: 83
166 video per il periodo (effettivo) da 2017-11-13 00:00:00 a 2017-11-20 15:55:00

5)  ->
Num video CON cicloni: 61, Num video SENZA cicloni: 998	Totale video tiles: 105

In [31]:
print(sup_data_train.df_video.label.sum(), sup_data_train.df_video.shape[0])

1773 3546
